# Load Conf and Credentials

## Load Directory Locations

In [2]:
import json
import os

# Check if the file exists and load the JSON file into a dictionary
file_path = r'C:\Users\mike\Develop\Projects\Code Notebook\Credentials\locations_conf.json'
if os.path.exists(file_path):
    with open(file_path, 'r') as f:
        locations_data = json.load(f)
    print(locations_data)
else:
    print(f"File not found: {file_path}")

{'Common_Funcs_Dir': '/Users/mike/Develop/Projects/Code Notebook/Common/Functions', 'Credentials_Dir': '/Users/mike/Develop/Projects/Code Notebook/Credentials', 'Rel_Pickes_Dir': '../.pickles', 'Pub_Data_Dir': "'/Users/mike/Data/Public", 'BQ_Service_Key': '/Users/mike/Develop/Conf/GCP Service Keys/mikecancell-development-0bcca41f8486.json'}


# Load the Pickled Dataframes into memory

In [3]:
import os
import pickle
import pandas as pd
import zipfile

directory = locations_data['Rel_Pickes_Dir']

for filename in os.listdir(directory):
    if filename.endswith('.pkl') or filename.endswith('.pkl.zip'):
        filepath = os.path.join(directory, filename)
        try:
            if filename.endswith('.pkl.zip'):
                with zipfile.ZipFile(filepath, 'r') as zip_ref:
                    for zip_info in zip_ref.infolist():
                        if zip_info.filename.endswith('.pkl'):
                            with zip_ref.open(zip_info) as file:
                                df_name = os.path.splitext(os.path.splitext(filename)[0])[0]
                                globals()[df_name] = pd.DataFrame(pickle.load(file))
                                print(f"Loaded DataFrame from zip: {df_name}")
                                print(globals()[df_name].info())
            else:
                with open(filepath, 'rb') as file:
                    df_name = os.path.splitext(filename)[0]
                    globals()[df_name] = pd.DataFrame(pickle.load(file))
                    print(f"Loaded DataFrame: {df_name}")
                    print(globals()[df_name].info())
        except Exception as e:
            print(f"Error loading {filename}: {e}")

# Now each pickle file is loaded into its own respective DataFrame variable

Loaded DataFrame from zip: crime_facts
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972000 entries, 0 to 971999
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   dr_no           972000 non-null  object        
 1   date_rptd       972000 non-null  datetime64[ns]
 2   datetime_occ    972000 non-null  datetime64[ns]
 3   rpt_dist_no     972000 non-null  int16         
 4   vict_age        972000 non-null  int16         
 5   lat             972000 non-null  float64       
 6   lon             972000 non-null  float64       
 7   area            972000 non-null  int16         
 8   premis_cd       972000 non-null  int16         
 9   crm_cd          972000 non-null  int16         
 10  vict_sex        972000 non-null  category      
 11  vict_descent    972000 non-null  category      
 12  weapon_used_cd  972000 non-null  int16         
 13  status          972000 non-null  category      
dt

# Load the Retrieved Dataframes into Individual CSV Files

In [4]:
# Define the public directory path
public_directory = locations_data['Pub_Data_Dir']

# Ensure the directory exists
os.makedirs(public_directory, exist_ok=True)

# Save each dimension DataFrame to a CSV file
dimension_dfs = {
    'crime_facts': crime_facts,
    'dim_area': dim_area,
    'dim_census': dim_census,
    'dim_crime': dim_crime,
    'dim_location': dim_location,
    'dim_premise': dim_premise,
    'dim_status': dim_status,
    'dim_victim': dim_victim,
    'dim_weapon': dim_weapon
}

for df_name, df in dimension_dfs.items():
    csv_path = os.path.join(public_directory.strip("'"), f"{df_name}.csv")  # Remove extra quotes from the path
    df.to_csv(csv_path, index=False)
    if os.path.exists(csv_path):
        print(f"Saved {df_name} to {csv_path}")
    else:
        print(f"Failed to save {df_name} to {csv_path}")

Saved crime_facts to /Users/mike/Data/Public\crime_facts.csv
Saved dim_area to /Users/mike/Data/Public\dim_area.csv
Saved dim_census to /Users/mike/Data/Public\dim_census.csv
Saved dim_crime to /Users/mike/Data/Public\dim_crime.csv
Saved dim_location to /Users/mike/Data/Public\dim_location.csv
Saved dim_premise to /Users/mike/Data/Public\dim_premise.csv
Saved dim_status to /Users/mike/Data/Public\dim_status.csv
Saved dim_victim to /Users/mike/Data/Public\dim_victim.csv
Saved dim_weapon to /Users/mike/Data/Public\dim_weapon.csv
